In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments
import evaluate
import numpy as np
from transformers import Trainer

In [4]:
raw_datasets = load_dataset("glue","mrpc")
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels = 2)
training_args = TrainingArguments("test-trainer",eval_strategy="epoch",fp16=True,)

def tokenized_function(example):
     return tokenizer(example["sentence1"],example["sentence2"],truncation = True)


tokenized_datasets = raw_datasets.map(tokenized_function,batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics(eval_preds):
     metric = evaluate.load("glue","mrpc")
     logits, labels = eval_preds
     predictions = np.argmax(logits,axis=-1)
     return metric.compute(predictions = predictions,references = labels) 

trainer = Trainer(
     model,
     training_args,
     data_collator,
     tokenized_datasets["train"],
     tokenized_datasets["validation"],
     tokenizer = tokenizer,
     compute_metrics=compute_metrics,
)
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\62453\AppData\Local\Temp\ipykernel_49232\642593929.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.590242,0.696078,0.818182
2,0.543200,0.425100,0.848039,0.890845
3,0.357800,0.597572,0.848039,0.892734


Using the latest cached version of the module from C:\Users\62453\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--glue\05234ba7acc44554edcca0978db5fa3bc600eeee66229abe79ff9887eacaf3ed (last modified on Sun Dec 21 20:53:34 2025) since it couldn't be found locally at evaluate-metric--glue, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from C:\Users\62453\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--glue\05234ba7acc44554edcca0978db5fa3bc600eeee66229abe79ff9887eacaf3ed (last modified on Sun Dec 21 20:53:34 2025) since it couldn't be found locally at evaluate-metric--glue, or remotely on the Hugging Face Hub.


TrainOutput(global_step=1377, training_loss=0.3863482949693911, metrics={'train_runtime': 112.5967, 'train_samples_per_second': 97.729, 'train_steps_per_second': 12.229, 'total_flos': 419446300011600.0, 'train_loss': 0.3863482949693911, 'epoch': 3.0})